### This Jupyter notebook will be used to determine the user defines variables

#### Step 1 - Retrieve Location of Desired Study Area
Using either a shapefile, coordinate, or WKT the study area for where satalite imagery will be downloaded is to be determined

In [28]:
#import requirements
import pandas as pd
import geopandas as gpd
import asf_search as asf
from shapely import wkt

### Find data using a geographic search
To begin, we will need to craft a suitable WKT. ASF's [Vertex](https://search.asf.alaska.edu) can be helpful in this regard, as it allows you to draw on a map, or import a geospatial file such as a shapefile or geojson, after which a WKT string can be copied and used elsewhere.

In [29]:
#Select location using a WKT obtained from ASF's Vertex search site or copied from a geodatabase (must resemble: 'POLYGON((-85.5523 64.8284,-84.9823 64.8284,-84.9823 65.3214,-85.5523 65.3214,-85.5523 64.8284))')

location = input('input WKT string')
print('The given location is: ' + location)

The given location is: POLYGON((-85.5523 64.8284,-84.9823 64.8284,-84.9823 65.3214,-85.5523 65.3214,-85.5523 64.8284))


In [35]:
#input date range for search
dateStart = input('input start date in YYYY-MM-DD')
dateEnd = input('input end date in YYYY-MM-DD')

print('The date range of the search will be: ' + dateStart + ' to ' + dateEnd)

The date range of the search will be: 2020-01-01 to 2020-01-08


In [36]:
#perform the search for results with the given locations centroid and date range

centroidwkt = wkt.loads(location)
centroid = centroidwkt.centroid.wkt


centroid_results = asf.geo_search(
        intersectsWith=centroid,
        platform=asf.PLATFORM.SENTINEL1,
        maxResults=50,
        start=dateStart,
        end=dateEnd)

print(f'{len(centroid_results)} results found')

22 results found


In [37]:
#perform the search for results with the given location and date range, use if centroid yields no results

geo_results = asf.geo_search(
        intersectsWith=location,
        platform=asf.PLATFORM.SENTINEL1,
        maxResults=50,
        start=dateStart,
        end=dateEnd)

print(f'{len(geo_results)} results found')

26 results found


In [42]:
geo_results[1]

### Finding data using an existing product name
If you already have the name of the product (i.e. ')